In [43]:
import numpy as np
import pandas as pd
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, precision_recall_fscore_support
from scipy.stats import chi2_contingency

In [44]:
df1 = pd.read_excel(r"C:\Users\prana\OneDrive\Desktop\credit risk_ml\Internal_Bank_Dataset.xlsx")


In [45]:
df1

,PROSPECTID,Total_TL,Tot_Closed_TL,Tot_Active_TL,Total_TL_opened_L6M,Tot_TL_closed_L6M,pct_tl_open_L6M,pct_tl_closed_L6M,pct_active_tl,pct_closed_tl,...,CC_TL,Consumer_TL,Gold_TL,Home_TL,PL_TL,Secured_TL,Unsecured_TL,Other_TL,Age_Oldest_TL,Age_Newest_TL
0,1,5,4,1,0,0,0.000,0.00,0.200,0.800,...,0,0,1,0,4,1,4,0,72,18
1,2,1,0,1,0,0,0.000,0.00,1.000,0.000,...,0,1,0,0,0,0,1,0,7,7
2,3,8,0,8,1,0,0.125,0.00,1.000,0.000,...,0,6,1,0,0,2,6,0,47,2
3,4,1,0,1,1,0,1.000,0.00,1.000,0.000,...,0,0,0,0,0,0,1,1,5,5
4,5,3,2,1,0,0,0.000,0.00,0.333,0.667,...,0,0,0,0,0,3,0,2,131,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51331,51332,3,0,3,1,0,0.333,0.00,1.000,0.000,...,0,2,0,0,0,0,3,1,24,5
51332,51333,4,2,2,0,1,0.000,0.25,0.500,0.500,...,0,2,0,0,0,2,2,0,74,7
51333,51334,2,1,1,1,1,0.500,0.50,0.500,0.500,...,0,2,0,0,0,0,2,0,9,5
51334,51335,2,1,1,0,0,0.000,0.00,0.500,0.500,...,0,2,0,0,0,0,2,0,15,8


In [46]:
df2= pd.read_excel(r"c:\Users\prana\OneDrive\Desktop\credit risk_ml\External_Cibil_Dataset.xlsx")


In [47]:
df2

,PROSPECTID,time_since_recent_payment,time_since_first_deliquency,time_since_recent_deliquency,num_times_delinquent,max_delinquency_level,max_recent_level_of_deliq,num_deliq_6mts,num_deliq_12mts,num_deliq_6_12mts,...,pct_CC_enq_L6m_of_L12m,pct_PL_enq_L6m_of_ever,pct_CC_enq_L6m_of_ever,max_unsec_exposure_inPct,HL_Flag,GL_Flag,last_prod_enq2,first_prod_enq2,Credit_Score,Approved_Flag
0,1,549,35,15,11,29,29,0,0,0,...,0.0,0.0,0.0,13.333,1,0,PL,PL,696,P2
1,2,47,-99999,-99999,0,-99999,0,0,0,0,...,0.0,0.0,0.0,0.860,0,0,ConsumerLoan,ConsumerLoan,685,P2
2,3,302,11,3,9,25,25,1,9,8,...,0.0,0.0,0.0,5741.667,1,0,ConsumerLoan,others,693,P2
3,4,-99999,-99999,-99999,0,-99999,0,0,0,0,...,0.0,0.0,0.0,9.900,0,0,others,others,673,P2
4,5,583,-99999,-99999,0,-99999,0,0,0,0,...,0.0,0.0,0.0,-99999.000,0,0,AL,AL,753,P1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51331,51332,15,24,23,2,24,24,0,0,0,...,0.0,0.0,0.0,1.661,0,0,ConsumerLoan,ConsumerLoan,650,P4
51332,51333,57,-99999,-99999,0,-99999,0,0,0,0,...,0.0,0.0,0.0,0.520,0,0,others,others,702,P1
51333,51334,32,-99999,-99999,0,-99999,0,0,0,0,...,0.0,1.0,0.0,0.567,0,0,ConsumerLoan,others,661,P3
51334,51335,58,-99999,-99999,0,-99999,0,0,0,0,...,0.0,0.0,0.0,1.202,0,0,ConsumerLoan,others,686,P2


In [48]:
columns_to_be_removed = []

for i in df2.columns:
    if df2.loc[df2[i] == -99999].shape[0] > 10000:
        columns_to_be_removed .append(i)

In [49]:
print(columns_to_be_removed)

['time_since_first_deliquency', 'time_since_recent_deliquency', 'max_delinquency_level', 'max_deliq_6mts', 'max_deliq_12mts', 'CC_utilization', 'PL_utilization', 'max_unsec_exposure_inPct']


In [50]:
df2 = df2.drop(columns_to_be_removed, axis =1)

In [51]:
columns_to_be_removed = []

for i in df1.columns:
    if df1.loc[df1[i] == -99999].shape[0] > 10000:
        columns_to_be_removed .append(i)

In [52]:
print(columns_to_be_removed)

[]


In [53]:
for i in df2.columns:
    df2 = df2.loc[ df2[i] != -99999 ]

In [54]:
for i in df1.columns:
    df1 = df1.loc[ df1[i] != -99999 ]

In [55]:
for i in list(df1.columns):
    if i in list(df2.columns):
        print (i)

PROSPECTID


In [56]:
df = pd. merge ( df1, df2, how ='inner', left_on = ['PROSPECTID'], right_on = ['PROSPECTID'] )

In [57]:
df

,PROSPECTID,Total_TL,Tot_Closed_TL,Tot_Active_TL,Total_TL_opened_L6M,Tot_TL_closed_L6M,pct_tl_open_L6M,pct_tl_closed_L6M,pct_active_tl,pct_closed_tl,...,pct_PL_enq_L6m_of_L12m,pct_CC_enq_L6m_of_L12m,pct_PL_enq_L6m_of_ever,pct_CC_enq_L6m_of_ever,HL_Flag,GL_Flag,last_prod_enq2,first_prod_enq2,Credit_Score,Approved_Flag
0,1,5,4,1,0,0,0.000,0.00,0.200,0.800,...,0.0,0.0,0.000,0.0,1,0,PL,PL,696,P2
1,2,1,0,1,0,0,0.000,0.00,1.000,0.000,...,0.0,0.0,0.000,0.0,0,0,ConsumerLoan,ConsumerLoan,685,P2
2,3,8,0,8,1,0,0.125,0.00,1.000,0.000,...,0.0,0.0,0.000,0.0,1,0,ConsumerLoan,others,693,P2
3,5,3,2,1,0,0,0.000,0.00,0.333,0.667,...,0.0,0.0,0.000,0.0,0,0,AL,AL,753,P1
4,6,6,5,1,0,0,0.000,0.00,0.167,0.833,...,1.0,0.0,0.429,0.0,1,0,ConsumerLoan,PL,668,P3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42059,51332,3,0,3,1,0,0.333,0.00,1.000,0.000,...,0.0,0.0,0.000,0.0,0,0,ConsumerLoan,ConsumerLoan,650,P4
42060,51333,4,2,2,0,1,0.000,0.25,0.500,0.500,...,0.0,0.0,0.000,0.0,0,0,others,others,702,P1
42061,51334,2,1,1,1,1,0.500,0.50,0.500,0.500,...,1.0,0.0,1.000,0.0,0,0,ConsumerLoan,others,661,P3
42062,51335,2,1,1,0,0,0.000,0.00,0.500,0.500,...,0.0,0.0,0.000,0.0,0,0,ConsumerLoan,others,686,P2


In [58]:
for i in df.columns:
    if df[i].dtype == 'object':
        print(i)

MARITALSTATUS
EDUCATION
GENDER
last_prod_enq2
first_prod_enq2
Approved_Flag


In [59]:
for i in ['MARITALSTATUS', 'EDUCATION', 'GENDER', 'last_prod_enq2', 'first_prod_enq2']:
    chi2, pval, _, _ = chi2_contingency(pd.crosstab(df[i], df['Approved_Flag']))
    print(i, '---', pval)

MARITALSTATUS --- 3.578180861038862e-233
EDUCATION --- 2.6942265249737532e-30
GENDER --- 1.907936100186563e-05
last_prod_enq2 --- 0.0
first_prod_enq2 --- 7.84997610555419e-287


Since all the categorical features have pval <=0.05, we will accept all

In [60]:
numeric_columns = []
for i in df.columns:
    if df[i].dtype != 'object' and i not in ['PROSPECTID','Approved_Flag']:
        numeric_columns.append(i)

In [61]:
len(numeric_columns)

72

In [62]:
vif_data = df[numeric_columns]
total_columns = vif_data.shape[1]
columns_to_be_kept = []
column_index = 0

In [63]:
for i in range (0,total_columns):
    
    vif_value = variance_inflation_factor(vif_data, column_index)  
    if vif_value <= 6:
        columns_to_be_kept.append( numeric_columns[i] )
        column_index = column_index+1
    
    else:
        print(numeric_columns[i])
        vif_data = vif_data.drop([ numeric_columns[i] ] , axis=1)

c:\Users\prana\Desktop\New folder\Lib\site-packages\statsmodels\stats\outliers_influence.py:198: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


Total_TL


c:\Users\prana\Desktop\New folder\Lib\site-packages\statsmodels\stats\outliers_influence.py:198: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


Tot_Closed_TL
Tot_Active_TL
Total_TL_opened_L6M
Tot_TL_closed_L6M


c:\Users\prana\Desktop\New folder\Lib\site-packages\statsmodels\stats\outliers_influence.py:198: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


pct_active_tl
pct_closed_tl
Total_TL_opened_L12M
pct_tl_open_L12M


c:\Users\prana\Desktop\New folder\Lib\site-packages\statsmodels\stats\outliers_influence.py:198: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


Auto_TL
Consumer_TL
Gold_TL
num_times_delinquent


c:\Users\prana\Desktop\New folder\Lib\site-packages\statsmodels\stats\outliers_influence.py:198: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


num_deliq_6mts
num_deliq_12mts
num_times_30p_dpd
num_std
num_std_6mts
num_dbt_6mts
num_lss_6mts
tot_enq
CC_enq
CC_enq_L6m
PL_enq
PL_enq_L6m
enq_L12m
enq_L6m
AGE
pct_of_active_TLs_ever
pct_opened_TLs_L6m_of_L12m
pct_PL_enq_L6m_of_L12m
pct_CC_enq_L6m_of_L12m
Credit_Score


In [64]:
features=['MARITALSTATUS', 'EDUCATION', 'GENDER', 'last_prod_enq2', 'first_prod_enq2']+columns_to_be_kept+['Approved_Flag']

In [65]:
df=df[features]

In [66]:
df

,MARITALSTATUS,EDUCATION,GENDER,last_prod_enq2,first_prod_enq2,pct_tl_open_L6M,pct_tl_closed_L6M,Tot_TL_closed_L12M,pct_tl_closed_L12M,Tot_Missed_Pmnt,...,NETMONTHLYINCOME,Time_With_Curr_Empr,pct_currentBal_all_TL,CC_Flag,PL_Flag,pct_PL_enq_L6m_of_ever,pct_CC_enq_L6m_of_ever,HL_Flag,GL_Flag,Approved_Flag
0,Married,12TH,M,PL,PL,0.000,0.00,0,0.000,0,...,51000,114,0.798,0,1,0.000,0.0,1,0,P2
1,Single,GRADUATE,F,ConsumerLoan,ConsumerLoan,0.000,0.00,0,0.000,0,...,19000,50,0.370,0,0,0.000,0.0,0,0,P2
2,Married,SSC,M,ConsumerLoan,others,0.125,0.00,0,0.000,1,...,18,191,0.585,0,0,0.000,0.0,1,0,P2
3,Married,POST-GRADUATE,M,AL,AL,0.000,0.00,0,0.000,0,...,15000,75,0.000,0,0,0.000,0.0,0,0,P1
4,Married,12TH,M,ConsumerLoan,PL,0.000,0.00,1,0.167,0,...,0,154,0.000,0,0,0.429,0.0,1,0,P3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42059,Married,12TH,M,ConsumerLoan,ConsumerLoan,0.333,0.00,0,0.000,0,...,18500,249,0.230,0,0,0.000,0.0,0,0,P4
42060,Married,SSC,M,others,others,0.000,0.25,1,0.250,0,...,25000,186,0.660,0,0,0.000,0.0,0,0,P1
42061,Married,SSC,M,ConsumerLoan,others,0.500,0.50,1,0.500,0,...,18000,66,0.428,0,0,1.000,0.0,0,0,P3
42062,Single,UNDER GRADUATE,F,ConsumerLoan,others,0.000,0.00,1,0.500,0,...,12802,54,0.143,0,0,0.000,0.0,0,0,P2


In [67]:
df.loc[df['EDUCATION'] == 'SSC',['EDUCATION']]              = 1
df.loc[df['EDUCATION'] == '12TH',['EDUCATION']]             = 2
df.loc[df['EDUCATION'] == 'GRADUATE',['EDUCATION']]         = 3
df.loc[df['EDUCATION'] == 'UNDER GRADUATE',['EDUCATION']]   = 3
df.loc[df['EDUCATION'] == 'POST-GRADUATE',['EDUCATION']]    = 4
df.loc[df['EDUCATION'] == 'OTHERS',['EDUCATION']]           = 1
df.loc[df['EDUCATION'] == 'PROFESSIONAL',['EDUCATION']]     = 3

In [68]:
df_encoded = pd.get_dummies(df, columns=['MARITALSTATUS', 'GENDER', 'last_prod_enq2', 'first_prod_enq2'])

In [102]:
df_encoded['EDUCATION']

0        2
1        3
2        1
3        4
4        2
        ..
42059    2
42060    1
42061    1
42062    3
42063    1
Name: EDUCATION, Length: 42064, dtype: object

In [103]:
df_encoded['EDUCATION'] = df_encoded['EDUCATION'].astype(int)

In [77]:
pip install scikit-learn


Note: you may need to restart the kernel to use updated packages.


In [78]:
from sklearn.tree import DecisionTreeClassifier

In [104]:
y = df_encoded['Approved_Flag']
x = df_encoded. drop ( ['Approved_Flag'], axis = 1 )

In [105]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [81]:
dt_model = DecisionTreeClassifier(max_depth=20, min_samples_split=10)
dt_model.fit(x_train, y_train)
y_pred = dt_model.predict(x_test)

In [82]:
accuracy = accuracy_score(y_test, y_pred)
print ()
print(f"Accuracy: {accuracy:.2f}")
print ()

precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_pred)

for i, v in enumerate(['p1', 'p2', 'p3', 'p4']):
    print(f"Class {v}:")
    print(f"Precision: {precision[i]}")
    print(f"Recall: {recall[i]}")
    print(f"F1 Score: {f1_score[i]}")
    print()


Accuracy: 0.71

Class p1:
Precision: 0.721327967806841
Recall: 0.7071005917159763
F1 Score: 0.7141434262948207

Class p2:
Precision: 0.8066447749661967
Recall: 0.8277502477700693
F1 Score: 0.8170612404617491

Class p3:
Precision: 0.34717868338557994
Recall: 0.33433962264150946
F1 Score: 0.3406382160707421

Class p4:
Precision: 0.6521739130434783
Recall: 0.6122448979591837
F1 Score: 0.631578947368421



In [74]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
rf_classifier = RandomForestClassifier(n_estimators = 200, random_state=42)
rf_classifier.fit(x_train, y_train)
y_pred = rf_classifier.predict(x_test)

In [75]:
accuracy = accuracy_score(y_test, y_pred)
print ()
print(f'Accuracy: {accuracy}')
print ()
precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_pred)


for i, v in enumerate(['p1', 'p2', 'p3', 'p4']):
    print(f"Class {v}:")
    print(f"Precision: {precision[i]}")
    print(f"Recall: {recall[i]}")
    print(f"F1 Score: {f1_score[i]}")
    print()


Accuracy: 0.7642933555212171

Class p1:
Precision: 0.8364928909952607
Recall: 0.6962524654832347
F1 Score: 0.759956942949408

Class p2:
Precision: 0.7951359567640601
Recall: 0.933201189296333
F1 Score: 0.8586540215210651

Class p3:
Precision: 0.4507269789983845
Recall: 0.21056603773584906
F1 Score: 0.28703703703703703

Class p4:
Precision: 0.7162293488824101
Recall: 0.7162293488824101
F1 Score: 0.7162293488824101



In [84]:
from sklearn.ensemble import AdaBoostClassifier

In [87]:

adaboost_model = AdaBoostClassifier(n_estimators=50, learning_rate=1.0, random_state=42)
adaboost_model.fit(x_train, y_train)
y_pred = adaboost_model.predict(x_test)

In [88]:
accuracy = accuracy_score(y_test, y_pred)
print ()
print(f'Accuracy: {accuracy}')
print ()
precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_pred)


for i, v in enumerate(['p1', 'p2', 'p3', 'p4']):
    print(f"Class {v}:")
    print(f"Precision: {precision[i]}")
    print(f"Recall: {recall[i]}")
    print(f"F1 Score: {f1_score[i]}")
    print()




Accuracy: 0.7452751693807204

Class p1:
Precision: 0.7821782178217822
Recall: 0.7790927021696252
F1 Score: 0.7806324110671936

Class p2:
Precision: 0.7728774364056822
Recall: 0.9274529236868186
F1 Score: 0.8431390215334715

Class p3:
Precision: 0.384
Recall: 0.036226415094339624
F1 Score: 0.06620689655172414

Class p4:
Precision: 0.6151960784313726
Recall: 0.7317784256559767
F1 Score: 0.6684420772303595



In [90]:
pip install xgboost


  Obtaining dependency information for xgboost from https://files.pythonhosted.org/packages/e2/7b/8c1b410cd0604cee9a167a19f7e1746f5b92ae7d02ad574ab560b73c5a48/xgboost-2.1.1-py3-none-win_amd64.whl.metadata
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.1/124.9 MB 919.0 kB/s eta 0:02:16
   ---------------------------------------- 0.1/124.9 MB 1.0 MB/s eta 0:02:01
   ---------------------------------------- 0.2/124.9 MB 1.2 MB/s eta 0:01:40
   ---------------------------------------- 0.3/124.9 MB 1.3 MB/s eta 0:01:36
   ---------------------------------------- 0.4/124.9 MB 1.4 MB/s eta 0:01:30
   ---------------------------------------- 0.4/124.9 MB 1.3 MB/s eta 0:01:33
   ---------------------------------------- 0.5/124.9 MB 1.5 MB/s eta 0:01:25
   ---------------------------------------- 0.6/124.9 MB 1.5 MB/s eta 0:01:23
   ------------------------

In [106]:
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

xgb_classifier = xgb.XGBClassifier(objective='multi:softmax',  num_class=4)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
x_train, x_test, y_train, y_test = train_test_split(x,y_encoded,test_size=0.2, random_state=42)

In [107]:
y_encoded

array([1, 1, 1, ..., 2, 1, 1])

In [108]:
xgb_classifier.fit(x_train, y_train)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None, num_class=4,
              num_parallel_tree=None, ...)

In [109]:
y_pred = xgb_classifier.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.77


In [110]:
precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_pred)


for i, v in enumerate(['p1', 'p2', 'p3', 'p4']):
    print(f"Class {v}:")
    print(f"Precision: {precision[i]}")
    print(f"Recall: {recall[i]}")
    print(f"F1 Score: {f1_score[i]}")
    print()

Class p1:
Precision: 0.8275862068965517
Recall: 0.757396449704142
F1 Score: 0.7909371781668384

Class p2:
Precision: 0.8219885958660014
Recall: 0.914370664023786
F1 Score: 0.8657220606174346

Class p3:
Precision: 0.4447004608294931
Recall: 0.29132075471698116
F1 Score: 0.35202918376652986

Class p4:
Precision: 0.7283582089552239
Recall: 0.7113702623906706
F1 Score: 0.7197640117994101

